In [10]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
from lifelines import KaplanMeierFitter
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import gaussian_kde
from scipy.optimize import minimize
import time
from lifelines import KaplanMeierFitter
import pickle

In [11]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
from lifelines import KaplanMeierFitter
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import gaussian_kde
from scipy.optimize import minimize
import time
from scipy.stats import gamma, norm
from scipy.optimize import minimize
import time      

In [12]:
def fit_gamma_marginal(df, x0=[2.0, 1.5, 2.0, 1.0, 0.1], max_time: int = 60):
    """Fit Gamma marginals + Gaussian copula by MLE with served/abandon censoring semantics."""

    w = df['Waiting Time'].to_numpy()
    s = df['Handle Time'].to_numpy()
    delta = df['Abandon'].to_numpy()

    def neg_log_likelihood(theta, w, s, delta):
        shape_s, scale_s, shape_p, scale_p, rho = theta
        eps = 1e-12

        A = (delta == 1)                     # abandoned: observe P = w
        S = (delta == 0) & np.isfinite(s)    # served with valid s

        ll = 0.0

        if np.any(A):
            wA = np.maximum(w[A], eps)
            fp_log = gamma.logpdf(wA, a=shape_p, scale=scale_p)
            ll += fp_log.sum()

        if np.any(S):
            sS = np.maximum(s[S], eps)
            wS = np.maximum(w[S], eps)

            # log f_S(s)
            fs_log = gamma.logpdf(sS, a=shape_s, scale=scale_s)
            ll += fs_log.sum()

            # log P(P > w | S=s) under Gaussian copula; use logsf for stability
            u1 = np.clip(gamma.cdf(wS, a=shape_p, scale=scale_p), eps, 1 - eps)
            u2 = np.clip(gamma.cdf(sS, a=shape_s, scale=scale_s), eps, 1 - eps)
            z1, z2 = norm.ppf(u1), norm.ppf(u2)
            denom = np.sqrt(max(1.0 - rho**2, eps))
            t = (z1 - rho * z2) / denom
            ll += norm.logsf(t).sum()

        return -ll

    def gamma_moments(x):
        x = np.asarray(x)
        x = x[np.isfinite(x) & (x > 0)]
        if x.size == 0:
            return 1.0, 1.0
        m = x.mean()
        v = x.var(ddof=1) if x.size > 1 else (x.mean() ** 2)
        k = max((m * m) / max(v, 1e-8), 1e-3)
        th = max(v / max(m, 1e-8), 1e-3)
        return k, th

    # Init via moments
    k_p, th_p = gamma_moments(w[delta == 1])
    k_s, th_s = gamma_moments(s[delta == 0])

    x0 = np.array([k_s, th_s, k_p, th_p, 0.1], dtype=float)

    bounds = [
        (1e-4, None),   # shape_s
        (1e-4, None),   # scale_s
        (1e-4, None),   # shape_p
        (1e-4, None),   # scale_p
        (-0.999, 0.999) # rho
    ]

    res = minimize(neg_log_likelihood, x0=x0, args=(w, s, delta),
                   method='L-BFGS-B', bounds=bounds)

    shape_s_hat, scale_s_hat, shape_p_hat, scale_p_hat, rho_hat = res.x
    return  shape_p_hat, scale_p_hat, shape_s_hat, scale_s_hat, rho_hat


In [13]:
with open('params.pkl', 'rb') as f:
    params_all = pickle.load(f)

In [14]:
params_all

{1: {'shape_p': 5,
  'scale_p': 0.9,
  'shape_s': 4,
  'scale_s': 1,
  'arrival_rate': 7,
  'rho': 0.7},
 2: {'shape_p': 2,
  'scale_p': 5,
  'shape_s': 4,
  'scale_s': 1.5,
  'arrival_rate': 6,
  'rho': 0.7}}

In [15]:
num_contact_groups = 2
num_days = 1
macro_rep = 1
num_days_consider = 1 # This is what is going to be used. num_days is the max number of 
num_contact_groups = 2
rho_1, rho_2 = params_all[1]['rho'], params_all[2]['rho']
# _________
hours_of_operation = 9
hours_of_termination = 13
copula_params_dict = {1:[], 2:[]}
for i in range(1, macro_rep+1):
    df_train = pd.DataFrame()
    for j in range(1, num_days_consider+1):
        df_train_tmp = pd.read_csv(f'data_censored/Simulated_Contact_Macro_{i}_Day_{j}_Params_{rho_1}_{rho_2}.csv')
        df_train = pd.concat([df_train, df_train_tmp])
    for contact_group in range(1, num_contact_groups+1):
        df_train_contact = df_train[df_train['Contact Group']==contact_group]
        copula_params_tmp = fit_gamma_marginal(df_train_contact)
        copula_params_dict[contact_group].append(copula_params_tmp)




In [16]:
def sample_plot_copula(copula_params, num_samples: int = 1000, plot_figures= False):
    """Sample from fitted Gamma–Gaussian copula and plot scatter."""
    def _sample_gamma_gaussian_copula(n, shape_p, scale_p, shape_s, scale_s, rho=0.6, seed=None, eps=1e-12):
        rng = np.random.default_rng(seed)
        R = np.array([[1.0, rho], [rho, 1.0]])
        L = np.linalg.cholesky(R)
        Z = rng.standard_normal((n, 2))
        Y = Z @ L.T
        U = np.clip(norm.cdf(Y), eps, 1 - eps)
        P = gamma.ppf(U[:, 0], a=shape_p, scale=scale_p)
        S = gamma.ppf(U[:, 1], a=shape_s, scale=scale_s)
        return P, S

    alpha_s, scale_s, alpha_p, scale_p, rho = copula_params
    P, S = _sample_gamma_gaussian_copula(num_samples, alpha_p, scale_p, alpha_s, scale_s, rho, seed=42)

    if plot_figures==True:
        plt.figure(figsize=(7, 6))
        plt.scatter(P, S, alpha=0.45, s=12, c="steelblue", edgecolor='none')
        plt.xlabel("Patience Time (P)")
        plt.ylabel("Handle Time (S)")
        plt.title(f"Gamma–Gaussian Copula Samples (n={num_samples}, rho={rho:.3f})")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    return P, S

In [17]:
copula_params_dict

{1: [(6.463602002432687,
   0.5801567502655273,
   4.4986798076096335,
   0.8555048842659618,
   0.46027379593071577)],
 2: [(1.5185498126936283,
   6.168877237199344,
   4.677036422045809,
   1.2895432263220137,
   0.12685739805957763)]}

# Generate Datasets

In [18]:
bo_dataset = pd.read_csv(f'data_bo/Back_Office_Deadline.csv')
bo_dataset

,Contact Group,Deadline,Ratio
0,1,5,0.1
1,2,6,0.2


In [19]:
def sample_gamma_gaussian_copula(n, shape_p, scale_p, shape_s, scale_s, rho=0.6):
    L = np.array([[1.0, 0.0], [rho, np.sqrt(1.0 - rho**2)]])
    Z = np.random.standard_normal((n, 2))
    Y = Z @ L.T
    U = norm.cdf(Y)
    U = np.clip(U, 1e-12, 1 - 1e-12)
    P = gamma.ppf(U[:, 0], a=shape_p, scale=scale_p) 
    S = gamma.ppf(U[:, 1], a=shape_s, scale=scale_s)
    return P, S

In [20]:
macro_rep = 1
num_reps_simulate = 500 
num_contact_groups = 2
rho_1, rho_2 = params_all[1]['rho'], params_all[2]['rho']

In [21]:
macro_rep = macro_rep # so that you knwo they should be the same.
for i in range(1, macro_rep+1):
    for j in range(1, num_reps_simulate+1):
        df_tmp = pd.DataFrame()
        for contact_group in range(1, num_contact_groups+1):
            bo_ratio = bo_dataset.loc[bo_dataset['Contact Group'] == contact_group, 'Ratio'].iloc[0]
            bo_deadline = bo_dataset.loc[bo_dataset['Contact Group'] == contact_group, 'Deadline'].iloc[0]
            df_tmp_contact = pd.read_csv(f'data_sim_arrival/Contact_Sim_Group_{contact_group}_Macro_{i}_Rep_{j}_Params_{rho_1}_{rho_2}.csv')
            contact_params = copula_params_dict[contact_group][i-1]
            shape_p_hat, scale_p_hat, shape_s_hat, scale_s_hat, rho_hat = contact_params
            n_df = len(df_tmp_contact)
            P, S = sample_gamma_gaussian_copula(n_df, shape_p_hat, scale_p_hat, shape_s_hat, scale_s_hat, rho_hat)
            BO_P = np.random.uniform(0,1,size=n_df)
            df_tmp_contact['Patience Time'] = P 
            df_tmp_contact['Handle Time'] = S
            print(bo_ratio, bo_deadline)
            df_tmp_contact['BO Need'] = np.where(BO_P<bo_ratio, 1, 0)
            df_tmp_contact['BO Deadline'] = bo_deadline
            df_tmp = pd.concat([df_tmp, df_tmp_contact])
        df_tmp.to_csv(f'data_sim/Contact_Sim_Copula_Macro_{i}_Rep_{j}_Params_{rho_1}_{rho_2}.csv')        

0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 

# Generate RL Datasets

In [22]:
macro_rep = 1
num_reps_simulate = 50 
num_contact_groups = 2
rho_1, rho_2 = params_all[1]['rho'], params_all[2]['rho']

In [24]:
macro_rep = macro_rep # so that you knwo they should be the same.
for i in range(1, macro_rep+1):
    for j in range(1, num_reps_simulate+1):
        df_tmp = pd.DataFrame()
        for contact_group in range(1, num_contact_groups+1):
            bo_ratio = bo_dataset.loc[bo_dataset['Contact Group'] == contact_group, 'Ratio'].iloc[0]
            bo_deadline = bo_dataset.loc[bo_dataset['Contact Group'] == contact_group, 'Deadline'].iloc[0]
            df_tmp_contact = pd.read_csv(f'data_sim_arrival_rl/Contact_Sim_Group_{contact_group}_Macro_{i}_Rep_{j}_Params_{rho_1}_{rho_2}.csv')
            contact_params = copula_params_dict[contact_group][i-1]
            shape_p_hat, scale_p_hat, shape_s_hat, scale_s_hat, rho_hat = contact_params
            n_df = len(df_tmp_contact)
            P, S = sample_gamma_gaussian_copula(n_df, shape_p_hat, scale_p_hat, shape_s_hat, scale_s_hat, rho_hat)
            BO_P = np.random.uniform(0,1,size=n_df)
            df_tmp_contact['Patience Time'] = P 
            df_tmp_contact['Handle Time'] = S
            print(bo_ratio, bo_deadline)
            df_tmp_contact['BO Need'] = np.where(BO_P<bo_ratio, 1, 0)
            df_tmp_contact['BO Deadline'] = bo_deadline
            df_tmp = pd.concat([df_tmp, df_tmp_contact])
        df_tmp.to_csv(f'data_sim_rl/Contact_Sim_Copula_Macro_{i}_Rep_{j}_Params_{rho_1}_{rho_2}.csv')        

0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
0.1 5
0.2 6
